### Prueba 7 (RDD) Candidata

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, lower, explode, regexp_replace, size
import re
import math
import pandas as pd

spark = SparkSession.builder.appName("RecomendacionLibros").master("local[*]").getOrCreate()
sc = spark.sparkContext

# 1. Cargar documentos
rdd = sc.wholeTextFiles("Libros_clean/*.txt")
rdd = rdd.map(lambda x: (re.findall(r"[^/]+$", x[0])[0], x[1]))

# 2. Limpiar texto y pasar a minúsculas
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9 \n]', '', text)
    return text.lower()

rdd_clean = rdd.map(lambda x: (x[0], clean_text(x[1])))

# 3. Quitar stopwords
from pyspark.ml.feature import StopWordsRemover
stopwords = StopWordsRemover.loadDefaultStopWords("english")

rdd_filtrado = rdd_clean.map(lambda x: (x[0], " ".join([w for w in x[1].split() if w not in stopwords])))

# 4. TF crudo
rdd_cont = rdd_filtrado.flatMap(lambda x: [((x[0], w), 1) for w in x[1].split()]).reduceByKey(lambda a, b: a + b)

# 5. TF normalizado
rdd_total = rdd_cont.map(lambda x: (x[0][0], x[1])).reduceByKey(lambda a, b: a + b)

rdd_normalizado = rdd_cont.map(lambda x: (x[0][0], (x[0][1], x[1]))).join(rdd_total)\
                           .map(lambda x: ((x[0], x[1][0][0]), x[1][0][1] / x[1][1]))

# 6. DF (document frequency)
rdd_doc_unico = rdd_filtrado.flatMap(lambda x: [(w, x[0]) for w in set(x[1].split())])
rdd_df = rdd_doc_unico.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)

# 7. TF-IDF distribuido
N = rdd_filtrado.count()

rdd_df_map = rdd_df.collectAsMap()  # DF pequeño, cabe en driver
rdd_tfidf = rdd_normalizado.map(lambda x: ((x[0][0], x[0][1]), x[1] * math.log(N / rdd_df_map[x[0][1]])))

# 8. Crear vectores por documento (RDD distribuido)
rdd_vectores = rdd_tfidf.map(lambda x: (x[0][0], (x[0][1], x[1]))).groupByKey()\
                         .mapValues(lambda vals: dict(vals))

# 9. Función de similitud coseno
def coseno(v1, v2):
    palabras = set(v1.keys()).union(v2.keys())
    dot = sum(v1.get(p, 0) * v2.get(p, 0) for p in palabras)
    norm1 = math.sqrt(sum(v1.get(p, 0)**2 for p in palabras))
    norm2 = math.sqrt(sum(v2.get(p, 0)**2 for p in palabras))
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot / (norm1 * norm2)

# 10. Similitud distribuida
rdd_pairs = rdd_vectores.cartesian(rdd_vectores)\
                        .filter(lambda x: x[0][0] < x[1][0])  # Evita pares duplicados y self-similarity

rdd_simil = rdd_pairs.map(lambda x: ((x[0][0], x[1][0]), coseno(x[0][1], x[1][1])))
top_simil = rdd_simil.takeOrdered(10, key=lambda x: -x[1])

25/12/10 20:04:47 WARN Utils: Your hostname, gael-guzman-B550MH-3-0 resolves to a loopback address: 127.0.1.1; using 192.168.1.3 instead (on interface enp3s0)
25/12/10 20:04:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 20:04:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/10 20:04:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/10 20:05:04 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
                                                                                

In [2]:
# Lista de documentos ordenada
docs = sorted(rdd_vectores.keys().collect())

# Construir diccionario de filas
rdd_rows = rdd_vectores.cartesian(rdd_vectores)\
                       .map(lambda x: ((x[0][0], x[1][0]), coseno(x[0][1], x[1][1])))\
                       .map(lambda x: (x[0][0], (x[0][1], x[1])))\
                       .groupByKey()\
                       .mapValues(dict)
rows = rdd_rows.collect()

# Crear DataFrame asegurando el mismo orden para filas y columnas
data = {}
for doc, sims in rows:
    data[doc] = [sims.get(d, 0.0) for d in docs]

df = pd.DataFrame(data, index=docs, columns=docs)

#print(df)

In [10]:

# Convertimos la matriz de similitud RDD a DataFrame Pandas
docs = rdd_vectores.keys().collect()
sim_matrix = pd.DataFrame(0, index=docs, columns=docs, dtype=float)

for ((doc1, doc2), sim) in rdd_simil.collect():
    sim_matrix.loc[doc1, doc2] = sim
    sim_matrix.loc[doc2, doc1] = sim

# Colocar 1.0 en la diagonal
for d in docs:
    sim_matrix.loc[d, d] = 1.0

# Función 1️⃣: libros más parecidos
def libros_parecidos():
    doc = input("Ingrese el nombre del documento (ej: Doc1.txt): ")
    n = int(input("Ingrese la cantidad de libros parecidos que desea: "))
    
    if doc not in sim_matrix.index:
        print("Documento no encontrado.")
        return
    
    similitudes_doc = sim_matrix.loc[doc].drop(doc)  # Quitamos el propio doc
    top_docs = similitudes_doc.sort_values(ascending=False).head(n)
    print(f"\nLos {n} libros más parecidos a '{doc}' son:")
    print(top_docs)

vec_dict = rdd_vectores.collectAsMap()
# Función 2️⃣: cantidad de palabras que describen un documento
def cantidad_palabras():
    doc = input("Ingrese el nombre del documento (ej: Doc1.txt): ")
    n = int(input("Ingrese la cantidad de palabras que desea ver que describen el documento: "))
    
    if doc not in vec_dict:
        print("Documento no encontrado.")
        return
    
    vec = vec_dict[doc]  # Diccionario de palabras y TF-IDF
    
    # Ordenar las palabras por TF-IDF descendente y tomar las top n
    top_words = sorted(vec.items(), key=lambda x: x[1], reverse=True)[:n]
    
    print(f"\nLas {n} palabras que describen mejor el documento '{doc}' son:")
    for palabra, tfidf in top_words:
        print(f"{palabra} : {tfidf:.4f}")



In [11]:
# Ejemplo de uso:
#libros_parecidos()
cantidad_palabras()

Ingrese el nombre del documento (ej: Doc1.txt):  A_Tale_of_Two_Cities_by_Charles_Dickens.txt
Ingrese la cantidad de palabras que desea ver que describen el documento:  20



Las 20 palabras que describen mejor el documento 'A_Tale_of_Two_Cities_by_Charles_Dickens.txt' son:
lorry : 0.0207
defarge : 0.0203
manette : 0.0112
pross : 0.0108
darnay : 0.0101
carton : 0.0090
lucie : 0.0090
cruncher : 0.0078
stryver : 0.0075
tellsons : 0.0060
jerry : 0.0050
monseigneur : 0.0048
evrmonde : 0.0033
wineshop : 0.0032
madame : 0.0032
barsad : 0.0032
sydney : 0.0030
jacques : 0.0023
mender : 0.0023
lorrys : 0.0022


In [12]:
libros_parecidos()

Ingrese el nombre del documento (ej: Doc1.txt):  A_Tale_of_Two_Cities_by_Charles_Dickens.txt
Ingrese la cantidad de libros parecidos que desea:  20



Los 20 libros más parecidos a 'A_Tale_of_Two_Cities_by_Charles_Dickens.txt' son:
The_Adventures_of_Roderick_Random_by_T._Smollett.txt                                                       0.030836
The_Works_of_Edgar_Allan_Poe_—_Volume_2_by_Edgar_Allan_Poe.txt                                             0.025720
Jane_Eyre_An_Autobiography_by_Charlotte_Brontë.txt                                                         0.023761
The_Interesting_Narrative_of_the_Life_of_Olaudah_Equiano,_Or_Gustavus_Vassa,_The_African_by_Equiano.txt    0.023346
Romantic_castles_and_palaces_.txt                                                                          0.022054
Les_Misérables_by_Victor_Hugo.txt                                                                          0.021922
Ulysses_by_James_Joyce.txt                                                                                 0.021419
How_to_Observe_Morals_and_Manners_by_Harriet_Martineau.txt                                                

In [13]:
rdd_normalizado.take(3)

[(('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'christmas'),
  0.006421951397504196),
 (('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'carol'),
  0.00029190688170473617),
 (('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'story'),
  0.00029190688170473617)]